In [ ]:
# pyright: reportMissingImports=false
import sys
import os
import time
import logging
import datetime
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import requests
import pandas_gbq
from dreams_core.googlecloud import GoogleCloud as dgc
from dreams_core import core as dc
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema

# load dotenv
load_dotenv()

# configure logger
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s [%(name)s.%(funcName)s:%(lineno)d] %(message)s',
    datefmt='%d/%b/%Y %H:%M:%S'
    )
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

## import local files if necessary
# sys.path.append('../GitHub/core-functions/src/dreams_core')
# import googlecloud as dgc2
# importlib.reload(dgc2)

# sys.path.append('../GitHub/etl-pipelines/coin_wallet_metrics')
# import coin_wallet_met?rics as wm
# importlib.reload(wm)

# production-ready code

In [ ]:
def classify_price_periods(prices_df_full, params):
    """
    Classifies price periods as bullish, bearish, or consolidation based on provided thresholds,
    using a rolling average price to smooth out short-term spikes and avoid misclassification.

    Parameters:
    prices_df_full (pd.DataFrame): Full DataFrame containing price data.
    params (dict): Dictionary containing parameters for classification:
        - cutoff_date (str): The cutoff date after which records will be considered.
        - period_min_duration (int): The minimum length of a bullish or bearish period.
        - bull_threshold (float): The minimum percentage change to classify a period as bullish.
        - bear_threshold (float): The minimum percentage change to classify a period as bearish.
        - rolling_window_size (int): The window size for the rolling average to smooth the data.

    Returns:
    pd.DataFrame: DataFrame with classified periods.
    """
    # Unpack parameters
    cutoff_date = params.get('cutoff_date', '2014-01-01')
    period_min_duration = params.get('period_min_duration', 30)
    bull_threshold = params.get('bull_threshold', 0.5)
    bear_threshold = params.get('bear_threshold', -0.4)
    rolling_window_size = params.get('rolling_window_size', 5)

    # 1. Filter and prepare data
    prices_df = prices_df_full[prices_df_full['date'] >= cutoff_date].copy()

    prices_df['date'] = pd.to_datetime(prices_df['date'])
    prices_df.sort_values('date', inplace=True)
    prices_df.reset_index(drop=True, inplace=True)

    # 2. Identify local price maxima and minima
    n = period_min_duration

    prices_df['local_max'] = prices_df.iloc[argrelextrema(prices_df['price'].values, comparator=lambda x, y: x > y, order=n)[0]].date
    prices_df['local_min'] = prices_df.iloc[argrelextrema(prices_df['price'].values, comparator=lambda x, y: x < y, order=n)[0]].date

    prices_df['local_max'] = prices_df['date'].isin(prices_df['local_max'])
    prices_df['local_min'] = prices_df['date'].isin(prices_df['local_min'])

    # 3. Calculate the rolling average price
    prices_df['rolling_price'] = prices_df['price'].rolling(window=rolling_window_size, min_periods=1).mean()

    # 4. Calculate the % change between local min/max and the price n days prior and n days after using the rolling price
    prices_df['rolling_price_n_days_prior'] = prices_df['rolling_price'].shift(n)
    prices_df['pct_change_from_prior'] = ((prices_df['rolling_price'] - prices_df['rolling_price_n_days_prior']) / prices_df['rolling_price_n_days_prior'])
    prices_df['rolling_price_n_days_after'] = prices_df['rolling_price'].shift(-n)
    prices_df['pct_change_from_after'] = ((prices_df['rolling_price_n_days_after'] - prices_df['rolling_price']) / prices_df['rolling_price'])

    # 5. Classify periods
    prices_df['period'] = None

    for i in range(len(prices_df)):
        if prices_df['local_max'].iloc[i]:
            # Assess for bullish classification leading up to a local max
            if prices_df['pct_change_from_prior'].iloc[i] > bull_threshold:
                start_idx = max(0, i - n)
                prices_df.loc[start_idx:i, 'period'] = 'bullish'

            # Always assess for bearish classification after a local max
            if prices_df['pct_change_from_after'].iloc[i] < bear_threshold:
                prices_df.loc[i+1:i+n, 'period'] = 'bearish'

        if prices_df['local_min'].iloc[i]:
            # Assess for bearish classification leading up to a local min
            if prices_df['pct_change_from_prior'].iloc[i] < bear_threshold:
                start_idx = max(0, i - n)
                prices_df.loc[start_idx:i, 'period'] = 'bearish'

            # Always assess for bullish classification after a local min
            if prices_df['pct_change_from_after'].iloc[i] > bull_threshold:
                prices_df.loc[i+1:i+n, 'period'] = 'bullish'

    prices_df['period'].fillna('consolidation', inplace=True)

    return prices_df


def graph_price_action(prices_df):
    """
    Graph the price action of a given DataFrame with color-coded periods.

    Parameters:
    prices_df (pd.DataFrame): The DataFrame containing date, price, and period data.
                              Assumes 'date', 'price', and 'period' columns exist in the DataFrame.

    Returns:
    None
    """
    # Set colors for different periods
    colors = {'bullish': 'green', 'bearish': 'red', 'consolidation': '#D3D3D3'}

    # Map the period to colors
    prices_df['color'] = prices_df['period'].map(colors)

    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(21, 10))

    # Plot the price data with color coding based on the period
    ax.plot(prices_df['date'], prices_df['price'], color='blue', label='_nolegend_')

    # Apply color to line segments based on the period
    for i in range(len(prices_df) - 1):
        ax.plot(prices_df['date'].iloc[i-1:i+1], prices_df['price'].iloc[i-1:i+1], color=prices_df['color'].iloc[i])

    # # Set the y-axis to log scale
    # ax.set_yscale('log')

    # Add labels and title
    symbol = prices_df.reset_index(drop=True)['symbol'][0]

    ax.set_xlabel('Date')
    ax.set_ylabel('Price')
    ax.set_title(f'{symbol} Price with Periods Highlighted')

    # Add a legend
    handles = [plt.Line2D([0], [0], color=color, label=label) for label, color in colors.items()]
    ax.legend(handles=handles)

    # Show the plot
    plt.show()



In [ ]:
query_sql = '''
    select 'BTC' as symbol,* from analytics.bitcoin_prices
    '''
prices_df_full_btc = dgc().run_sql(query_sql)
print(prices_df_full_btc.info())
prices_df_full_btc.head()

In [ ]:
query_sql = '''
    with filter as (
    select cmd.coin_id
    ,count(date) as records
    from core.coin_market_data cmd
    group by 1
    )

    select c.symbol
    ,cmd.date
    ,cmd.price
    ,cmd.market_cap
    ,cmd.volume
    from core.coins c
    join core.coin_market_data cmd on cmd.coin_id = c.coin_id
    join filter f on f.coin_id = cmd.coin_id
    where f.records > 360
    '''
prices_df_full_alts = dgc().run_sql(query_sql)
print(prices_df_full_alts.info())
prices_df_full_alts.head()

# code drafting workspace

In [ ]:
# i = 0
s = prices_df_full_alts['symbol'].unique()[i]
prices_df_full = prices_df_full_alts[prices_df_full_alts['symbol']==s]
print(s)
i+=1

In [ ]:
# Check the exact pct_change_after_max for 2024-03-30
i = prices_df[prices_df['date'] == '2024-03-30'].index[0]
end_idx = min(len(prices_df) - 1, i + n)
pct_change_after_max = (prices_df['rolling_price'].iloc[end_idx] - prices_df['rolling_price'].iloc[i]) / prices_df['rolling_price'].iloc[i]

print(f"Pct change after 2024-03-30: {pct_change_after_max:.4f}")

In [ ]:
# s = 0
s+=1
symbol = prices_df_full_alts['symbol'].unique()[s]
prices_df_full = prices_df_full_alts[prices_df_full_alts['symbol']==symbol]
print(symbol)


params = {
    'cutoff_date': '2014-01-01',
    'period_min_duration': 30,
    'bull_threshold': 0.4,
    'bear_threshold': -0.22,
    'rolling_window_size': 2
}

prices_df = classify_price_periods(prices_df_full, params)
graph_price_action(prices_df)

In [ ]:
pd.set_option('display.max_rows', 100)

prices_df[(prices_df['date'] >= '2023-12-01') & (prices_df['date'] <= '2024-01-15')]
# prices_df[(prices_df['date'] >= '2024-03-01') & (prices_df['date'] <= '2024-06-01')]

In [ ]:
n = 14
bull_threshold = 0.3
bear_threshold = -0.2


# Identify local maxima and minima within n days on either side of the date
df['local_max'] = df.iloc[argrelextrema(df['price'].values, comparator=lambda x, y: x > y, order=n)[0]].date
df['local_min'] = df.iloc[argrelextrema(df['price'].values, comparator=lambda x, y: x < y, order=n)[0]].date

# Merging results with original DataFrame
df['local_max'] = df['date'].isin(df['local_max'])
df['local_min'] = df['date'].isin(df['local_min'])

# Calculate the % change between local min/max and the price n days prior
df['price_n_days_prior'] = df['price'].shift(n)
df['pct_change_from_prior'] = ((df['price'] - df['price_n_days_prior']) / df['price_n_days_prior'])

df.reset_index(inplace=True,drop=True)
df

In [ ]:
# Initialize the 'period' column with None or empty strings
df['period'] = None

# Iterate over the DataFrame to set the 'period' based on bullish conditions
for i in range(len(df)):
    if df['local_max'].iloc[i] and df['pct_change_from_prior'].iloc[i] > bull_threshold:
        # Set the 'period' as 'bullish' for the n days preceding the local_max date and the date itself
        start_idx = max(0, i - n)  # Ensure the start index is not less than 0
        df.loc[start_idx:i, 'period'] = 'bullish'

# Iterate over the DataFrame to set the 'period' based on bearish conditions
for i in range(len(df)):
    if df['local_min'].iloc[i] and df['pct_change_from_prior'].iloc[i] < bear_threshold:
        # Set the 'period' as 'bearish' for the n days preceding the local_min date and the date itself
        start_idx = max(0, i - n)  # Ensure the start index is not less than 0
        df.loc[start_idx:i, 'period'] = 'bearish'

# Fill empty values in the 'period' column with 'consolidation'
df['period'].fillna('consolidation', inplace=True)

# Display the updated DataFrame
df[['date', 'price', 'local_max', 'local_min', 'pct_change_from_prior', 'period']]
